# Configuring `Prompt Template` to customize input

- General conversation with generic responses
- Now we will have more control over convo and type of responses

Let's create an application that makes a joke from any word we give as input.
- No general conversation will be possible, only configured behaviour -> a joke AI.


## Import dependencies

In [1]:
var Bedrock = require('@langchain/community/llms/bedrock').Bedrock;
var ChatPromptTemplate = require('@langchain/core/prompts').ChatPromptTemplate;

## Instantiate the `model` client

In [2]:
var model = new Bedrock({
    model_id:'amazon.titan-text-express-v1',
    temperature: 1,
    maxTokenCount: 512,
    topP: 0.9,
    verbose: true
});

## Create prompt template

### `ChatPromptTemplate.fromTemplate`
- sets up `system` message instructing LLM for behaviour using NL.

In [3]:
var promptTemplate = ChatPromptTemplate.fromTemplate("You are a comendian. Give me a funny joke based on the following word {input}");

In essence a prompt template takes input and parses it in a format that can be chained / consumed into the rest of the components. Advanced usages later on. Output of the formatter is:

In [4]:
promptTemplate.format({input: 'EXAMPLE'}).then((output) => console.log(output));

Promise { <pending> }

## Create a chain (sidenote)

- `pipe`: Pass in waterfall style from one component to next the output as input

### Basic

```javascript
const outputA = funcA();
const outputB = funcB(outputA);
```

### Intermediate
```javascript
funcB(funcA())
```

### Advanced (piping)

In [5]:
class PipeableObject {
    constructor(initialValue = null) {
        this.value = initialValue;
    }

    pipe(func) {
        this.value = func(this.value);
        return this;
    }

    getValue() {
        return this.value;
    }
}

var obj = new PipeableObject(5);

obj.pipe(value => value * 2) // Multiply by 2
   .pipe(value => value + 3) // Add 3
   .pipe(value => value / 2); // Divide by 2
   
console.log(obj.getValue());

Human: You are a comendian. Give me a funny joke based on the following word EXAMPLE


6.5


## Create a chain (template)

- We send the prompt to the model (pipe it)

In [6]:
var chainTemplate = promptTemplate.pipe(model);

## Invoke LLM

- Given input we invoke LLM and get a joke for the given input word using prompt template

In [7]:
var input = 'dog';

In [8]:
chainTemplate.invoke({input}).then((output) => console.log(output));

Promise { <pending> }

## Create prompt template (messages)

### `ChatPromptTemplate.fromMessages`
- explicitly set up `system` message instructing LLM for behaviour using `imperative` NL.
- explicitly set up additional context and parameters.

In [9]:
var promptMessages = ChatPromptTemplate.fromMessages([
    ["system", "You are a comendian. Generate a joke based on the word provided by the user."],
    ["user", "{input}"]
]);


[llm/start] [1:llm:Bedrock] Entering LLM run with input: {
  "prompts": [
    "Human: You are a comendian. Give me a funny joke based on the following word dog"
  ]
}


## Create a chain (messages)

- We send the prompt to the model (pipe it)

In [10]:
var chainMessages = promptMessages.pipe(model);

## Invoke LLM

- Given input we invoke LLM and get a joke for the given input word using prompt messages

In [ ]:
chainMessages.invoke({input}).then((output) => console.log(output));

Promise { <pending> }

[llm/start] [1:llm:Bedrock] Entering LLM run with input: {
  "prompts": [
    "System: You are a comendian. Generate a joke based on the word provided by the user.\nHuman: dog"
  ]
}
[llm/end] [1:llm:Bedrock] [1.64s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "\nBot: What did the dog say after eating a bone?"
      }
    ]
  ]
}

Bot: What did the dog say after eating a bone?
[llm/end] [1:llm:Bedrock] [1.86s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "\nQ: Why don't dogs have dentists?\nA: Because they give them bones to chew on!"
      }
    ]
  ]
}

Q: Why don't dogs have dentists?
A: Because they give them bones to chew on!
